In [20]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import torch
import timm
from tqdm.notebook import tqdm

from tensorflow.keras.preprocessing.image import load_img

In [18]:
class image_operation(Dataset):
    def __init__ (self,data_dir,transform = None):
        self.data=ImageFolder(data_dir , transform = transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes 

In [5]:
dataset = image_operation("/kaggle/input/cards-image-datasetclassification/train")
length=len(dataset)  
print(f"lenght of the data is {length} and the classes present in the data are \t {dataset.classes}")

lenght of the data is 7624 and the classes present in the data are 	 ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two o

In [6]:
train_dir="/kaggle/input/cards-image-datasetclassification/train"
val_dir="/kaggle/input/cards-image-datasetclassification/valid"
target_to_class={v: k for k,v in ImageFolder(train_dir).class_to_idx.items()}
print(target_to_class)

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

In [7]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

train_dataset = image_operation(train_dir,transform)
val_dataset = image_operation(val_dir,transform)

In [8]:
train_data = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_data = DataLoader(val_dataset,batch_size=32,shuffle=True)

In [9]:
for images , labels in train_data:
    break

In [10]:
images.shape , labels.shape

(torch.Size([32, 3, 256, 256]), torch.Size([32]))

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class card_classifier(nn.Module):
    def __init__(self,num_classes=len(dataset.classes)):
        super(card_classifier,self).__init__()
        self.base_model = timm.create_model("efficientnet_b0",pretrained=True)
        self.features = nn.Sequential(*list (self.base_model.children())[:-1])

        self.classifier=nn.Linear(1280,num_classes)

    def forward(self,x):
        x=self.features(x)
        output=self.classifier(x)
        return output
model = card_classifier()
model.to(device)

card_classifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_

In [21]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

training_loss=[]
validation_loss=[]
epochs=5
for i in tqdm(range(epochs)):
    model.train()
    running_loss=0.0
    for images,labels in train_data:
        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()*labels.size(0)
    train_loss=running_loss/len(train_data.dataset)
    training_loss.append(train_loss)


    model.eval()
    running_loss=0.0
    with torch.no_grad():
        for images,labels in val_data:
            images,labels = images.to(device),labels.to(device)
            output=model(images)
            loss=criterion(output,labels)
            running_loss+=loss.item()*labels.size(0)
        val_loss=running_loss/len(val_data.dataset)
        validation_loss.append(val_loss)
    print(f"epochs :{i+1}/{epochs} - train loss = {train_loss :.2f} , validation loss = {val_loss :.2f}")

  0%|          | 0/5 [00:00<?, ?it/s]

epochs :1/5 - train loss = 0.15 , validation loss = 0.11
epochs :2/5 - train loss = 0.12 , validation loss = 0.04
epochs :3/5 - train loss = 0.09 , validation loss = 0.11
epochs :4/5 - train loss = 0.08 , validation loss = 0.06
epochs :5/5 - train loss = 0.06 , validation loss = 0.06
